# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [3]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-15e28017fb-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Association Football Club explorative search") 

Consider the following exploratory information need:

> investigate the association football clubs and find the main BGPs related to this sport. Compare clubs in countries and championships won. Explore also the participation of the countries in the FIFA World Cup.

**Important consideration: in wikidata, we are talking of ASSOCIATION FOOTBALL**

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q11571`| Cristiano Ronaldo       | node |





Also consider

```
wd:Q11571 ?p ?obj .
```

is the BGP to retrieve all **properties of Cristiano Ronaldo**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for association football club

2. Identify the BGP for association football league

3. Return for each north american country the number of association football clubs

4. Identify the BGP for the UEFA Champions League.

5. Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

6. Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

7. Identify the BGP of the FIFA World Cup

8. Consider countries who participated at the FIFA World Cup more than 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

9. Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).


## Task 1
Identify the BGP for association football club

In [6]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [2]:
#wd:Q11571	Cristiano Ronaldo	node
#checking the instances of each property
queryString = """
SELECT DISTINCT ?p ?name ?ins ?insname
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   ?obj wdt:P31 ?ins.
   # get the label
   ?ins sc:name ?insname.
   ?p sc:name ?name
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('ins', 'http://www.wikidata.org/entity/Q618779'), ('insname', 'award')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender'), ('ins', 'http://www.wikidata.org/entity/Q48264'), ('insname', 'gender identity')]
[('p', 'http://www.wikidata.org/prop/direct/P6379'), ('name', 'has works in the collection'), ('ins', 'http://www.wikidata.org/entity/Q3343298'), ('insname', 'non-departmental public body')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team'), ('ins', 'http://www.wikidata.org/entity/Q23759293'), ('insname', 'national Olympic football team')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed'), ('ins', 'http://www.wikidata.org/entity/Q1288568'), ('insname', 'modern language')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language'), ('ins', 'http://www.wikidata.org/en

In [4]:
# finding the teams for CR
#[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team'), ('ins', 'http://www.wikidata.org/entity/Q15944511'),
#('insname', 'association football team')]
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q11571 wdt:P54 ?p.
  
   # get the label
 
   ?p sc:name ?name.
 
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q3590758'), ('name', 'Portugal Olympic football team')]
[('p', 'http://www.wikidata.org/entity/Q1422'), ('name', 'Juventus F.C.')]
[('p', 'http://www.wikidata.org/entity/Q18656'), ('name', 'Manchester United F.C.')]
[('p', 'http://www.wikidata.org/entity/Q8682'), ('name', 'Real Madrid CF')]
[('p', 'http://www.wikidata.org/entity/Q1630430'), ('name', 'Portugal national under-21 football team')]
[('p', 'http://www.wikidata.org/entity/Q267245'), ('name', 'Portugal national association football team')]
[('p', 'http://www.wikidata.org/entity/Q75729'), ('name', 'Sporting CP')]
[('p', 'http://www.wikidata.org/entity/Q1772776'), ('name', 'Portugal national under-20 football team')]
[('p', 'http://www.wikidata.org/entity/Q3590754'), ('name', 'Portugal national under-17 football team')]
[('p', 'http://www.wikidata.org/entity/Q21079208'), ('name', 'Portugal national under-15 football team')]
10


In [19]:
#[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team'),
#('ins', 'http://www.wikidata.org/entity/Q23759293'), ('insname', 'national Olympic football team')]
#wdt:P31	instance of	predicate

#checking the class of teams
queryString = """
SELECT DISTINCT ?p ?name ?res ?rname
WHERE {
   # bind something
   wd:Q11571 wdt:P54 ?p.
   ?p wdt:P31 ?res.
   # get the label
 
   ?p sc:name ?name.
   ?res sc:name ?rname
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q3590758'), ('name', 'Portugal Olympic football team'), ('res', 'http://www.wikidata.org/entity/Q23759293'), ('rname', 'national Olympic football team')]
[('p', 'http://www.wikidata.org/entity/Q1422'), ('name', 'Juventus F.C.'), ('res', 'http://www.wikidata.org/entity/Q15944511'), ('rname', 'association football team')]
[('p', 'http://www.wikidata.org/entity/Q18656'), ('name', 'Manchester United F.C.'), ('res', 'http://www.wikidata.org/entity/Q15944511'), ('rname', 'association football team')]
[('p', 'http://www.wikidata.org/entity/Q18656'), ('name', 'Manchester United F.C.'), ('res', 'http://www.wikidata.org/entity/Q476028'), ('rname', 'association football club')]
[('p', 'http://www.wikidata.org/entity/Q8682'), ('name', 'Real Madrid CF'), ('res', 'http://www.wikidata.org/entity/Q476028'), ('rname', 'association football club')]
[('p', 'http://www.wikidata.org/entity/Q1422'), ('name', 'Juventus F.C.'), ('res', 'http://www.wikidata.org/en

Final query for this task

In [5]:
#Final Result
queryString = """
SELECT DISTINCT ?res ?rname
WHERE {
   # bind something
   wd:Q11571 wdt:P54 ?p.
   ?p wdt:P31 ?res.
   # get the label
 
   ?p sc:name ?name.
   ?res sc:name ?rname
   filter (regex(?rname , 'association football'))
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('res', 'http://www.wikidata.org/entity/Q23847779'), ('rname', 'national under-23 association football team')]
[('res', 'http://www.wikidata.org/entity/Q476028'), ('rname', 'association football club')]
[('res', 'http://www.wikidata.org/entity/Q15944511'), ('rname', 'association football team')]
[('res', 'http://www.wikidata.org/entity/Q6979593'), ('rname', 'national association football team')]
[('res', 'http://www.wikidata.org/entity/Q23904673'), ('rname', 'national under-15 association football team')]
5


## Task 2
Identify the BGP for association football league

In [28]:
#wd:Q11571	Cristiano Ronaldo	node
#[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]

#FINDING LEAGUES OF CR 

queryString = """
SELECT DISTINCT ?res ?rname
WHERE {
   # bind something
   wd:Q11571 wdt:P118 ?res.

   
   # get the label
 
   ?res sc:name ?rname
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('res', 'http://www.wikidata.org/entity/Q182994'), ('rname', 'Liga Portugal')]
[('res', 'http://www.wikidata.org/entity/Q15804'), ('rname', 'Serie A')]
[('res', 'http://www.wikidata.org/entity/Q324867'), ('rname', 'La Liga')]
[('res', 'http://www.wikidata.org/entity/Q9448'), ('rname', 'Premier League')]
4


Final query for this task

In [ ]:
#FINAL QUERY 
#wd:Q11571	Cristiano Ronaldo	node
#[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
#wdt:P31	instance of	predicate

#FINDING CLASS OF LEAGUES 

queryString = """
SELECT DISTINCT ?res ?rname
WHERE {
   # bind something
   wd:Q11571 wdt:P118 ?d.
   ?d wdt:P31 ?res.

   
   # get the label
 
   #?p sc:name ?name.
   ?res sc:name ?rname
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('res', 'http://www.wikidata.org/entity/Q15991290'), ('rname', 'professional sports league')]
[('res', 'http://www.wikidata.org/entity/Q15991303'), ('rname', 'association football league')]
2


## Task 3
Return for each north american country the number of association football clubs

In [4]:
#[('res', 'http://www.wikidata.org/entity/Q476028'), ('rname', 'association football club')]

#FINDING association football club INSTANCES 

queryString = """
SELECT DISTINCT  ?obj ?objname
WHERE {
    
    ?obj wdt:P31 wd:Q476028 .
    
    ?obj sc:name ?objname .
   
}

LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4647107'), ('objname', 'A.D. Dili Leste')]
[('obj', 'http://www.wikidata.org/entity/Q8186475'), ('objname', 'Tahuichi Academy')]
[('obj', 'http://www.wikidata.org/entity/Q8208548'), ('objname', 'Atlético Trujillo')]
[('obj', 'http://www.wikidata.org/entity/Q2631783'), ('objname', 'Cizrespor')]
[('obj', 'http://www.wikidata.org/entity/Q3448775'), ('objname', 'FK Sloboda Mrkonjić Grad')]
[('obj', 'http://www.wikidata.org/entity/Q5100109'), ('objname', 'Chinandega FC')]
[('obj', 'http://www.wikidata.org/entity/Q5379014'), ('objname', 'Enkaspor')]
[('obj', 'http://www.wikidata.org/entity/Q7280157'), ('objname', "Radford Highlanders men's soccer")]
[('obj', 'http://www.wikidata.org/entity/Q7300846'), ('objname', 'Real Arroyo Seco')]
[('obj', 'http://www.wikidata.org/entity/Q14252413'), ('objname', 'Rapid JC')]
[('obj', 'http://www.wikidata.org/entity/Q14919844'), ('objname', "Fortuna '54")]
[('obj', 'http://www.wikidata.org/entity/Q3375656')

In [14]:
#FINDING PROPERTIES OF association football clubs
queryString = """
SELECT DISTINCT  ?p ?pname
WHERE {
    
    ?obj wdt:P31 wd:Q476028 .
    ?obj ?p ?res.
    
    ?p sc:name ?pname
   
}

LIMIT 20
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4421'), ('pname', 'Sportbox.ru ID')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pname', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pname', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1320'), ('pname', 'OpenCorporates ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pname', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('pname', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pname', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'h

In [7]:
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]

#FINDING THE COUNTRY OF EACH  association football club
queryString = """
SELECT DISTINCT  ?objname ?rname
WHERE {
    
    ?obj wdt:P31 wd:Q476028 ;
        wdt:P17 ?res.
    
    
    ?obj sc:name ?objname .
    ?res sc:name ?rname

   
}

LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('objname', 'Doğan Türk Birliği'), ('rname', 'Turkish Republic of Northern Cyprus')]
[('objname', 'Atlantico FC'), ('rname', 'Dominican Republic')]
[('objname', 'AS Pélican'), ('rname', 'Gabon')]
[('objname', 'Adouma FC'), ('rname', 'Gabon')]
[('objname', 'Delta Téléstar Gabon Télécom FC'), ('rname', 'Gabon')]
[('objname', 'Real de Banjul F.C.'), ('rname', 'The Gambia')]
[('objname', 'Horoya AC'), ('rname', 'Guinea')]
[('objname', 'Satellite FC'), ('rname', 'Guinea')]
[('objname', "Stella Club d'Adjamé"), ('rname', 'Ivory Coast')]
[('objname', 'Scorpion de Bey'), ('rname', 'Cameroon')]
10


In [10]:
#FINDING PROPERTIES OF COUNTRY TO SET THE CONTINENT
queryString = """
SELECT DISTINCT   ?p ?pname
WHERE {
    
    ?obj wdt:P31 wd:Q476028 ;
        wdt:P17 ?res.
    ?res ?p ?d.

    ?p sc:name ?pname.
  
   
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('pname', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('pname', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('pname', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1081'), ('pname', 'Human Development Index')]
[('p', 'http://www.wikidata.org/prop/direct/P1082'), ('pname', 'population')]
[('p', 'http://www.wikidata.org/prop/direct/P1138'), ('pname', 'Kunstindeks Danmark Artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pname', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1198'), ('pname', 'unemployment rate')]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('pname', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('pname', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('pnam

In [11]:
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]

#CHECKING THE CONTINENTS TO FINF NORTH AMERICA

queryString = """
SELECT DISTINCT   ?c ?cname
WHERE {
    
    ?team wdt:P31 wd:Q476028 ;
        wdt:P17 ?res.
    ?res wdt:P30 ?c.

    ?c sc:name ?cname .
    FILTER (REGEX(?cname, '[Nn]orth [Aa]merica'))
   
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q49'), ('cname', 'North America')]
1


Final query for this task

In [13]:
#[('c', 'http://www.wikidata.org/entity/Q49'), ('cname', 'North America')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
#[('res', 'http://www.wikidata.org/entity/Q476028'), ('rname', 'association football club')]


#FINAL QUERY 

queryString = """
SELECT DISTINCT   ?country ?label  (count(?team) as ?number)
WHERE {
    
    ?team wdt:P31 wd:Q476028 ;
        wdt:P17 ?country.
    ?country wdt:P30 wd:Q49.
 
 
    ?country sc:name ?label.

   
} group by  ?country ?label
"""
print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q754'), ('label', 'Trinidad and Tobago'), ('number', '22')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('label', 'Canada'), ('number', '138')]
[('country', 'http://www.wikidata.org/entity/Q778'), ('label', 'The Bahamas'), ('number', '45')]
[('country', 'http://www.wikidata.org/entity/Q11703'), ('label', 'United States Virgin Islands'), ('number', '1')]
[('country', 'http://www.wikidata.org/entity/Q786'), ('label', 'Dominican Republic'), ('number', '13')]
[('country', 'http://www.wikidata.org/entity/Q763'), ('label', 'Saint Kitts and Nevis'), ('number', '10')]
[('country', 'http://www.wikidata.org/entity/Q766'), ('label', 'Jamaica'), ('number', '30')]
[('country', 'http://www.wikidata.org/entity/Q790'), ('label', 'Haiti'), ('number', '66')]
[('country', 'http://www.wikidata.org/entity/Q1183'), ('label', 'Puerto Rico'), ('number', '1')]
[('country', 'http://www.wikidata.org/entity/Q34266'), ('label', 'Russian Empire'), ('numbe

## Task 4
Identify the BGP for the UEFA Champions League.

In [203]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http:

In [182]:
#[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
#[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
#[('p', 'http://www.wikidata.org/prop/direct/P2276'), ('name', 'UEFA player ID')]
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   # bind something
   wd:Q11571 wdt:P118 ?o.
   ?o sc:name ?name.
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)


Results
[('o', 'http://www.wikidata.org/entity/Q182994'), ('name', 'Liga Portugal')]
[('o', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A')]
[('o', 'http://www.wikidata.org/entity/Q324867'), ('name', 'La Liga')]
[('o', 'http://www.wikidata.org/entity/Q9448'), ('name', 'Premier League')]
4


In [184]:
#[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
#[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
#[('p', 'http://www.wikidata.org/prop/direct/P2276'), ('name', 'UEFA player ID')]
queryString = """
SELECT DISTINCT ?o ?name ?obj ?objname
WHERE {
   # bind something
   wd:Q11571 wdt:P118 ?o.
   ?o wdt:P279 ?obj .
   
   ?o sc:name ?name.
   ?obj sc:name ?objname.
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A'), ('obj', 'http://www.wikidata.org/entity/Q1478437'), ('objname', 'association football competition')]
[('o', 'http://www.wikidata.org/entity/Q324867'), ('name', 'La Liga'), ('obj', 'http://www.wikidata.org/entity/Q3270632'), ('objname', 'national championship')]
[('o', 'http://www.wikidata.org/entity/Q182994'), ('name', 'Liga Portugal'), ('obj', 'http://www.wikidata.org/entity/Q3270632'), ('objname', 'national championship')]
[('o', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A'), ('obj', 'http://www.wikidata.org/entity/Q3270632'), ('objname', 'national championship')]
4


In [187]:
#[('o', 'http://www.wikidata.org/entity/Q15804'), ('name', 'Serie A'),
#('obj', 'http://www.wikidata.org/entity/Q1478437'), ('objname', 'association football competition')]

queryString = """
SELECT DISTINCT  ?obj ?objname
WHERE {
   # bind something
   #wd:Q11571 wdt:P118 ?o.
   wd:Q1478437 wdt:P279 ?obj .
   
   #?o sc:name ?name.
   ?obj sc:name ?objname.
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)



Results
[('obj', 'http://www.wikidata.org/entity/Q16510064'), ('objname', 'sporting event')]
[('obj', 'http://www.wikidata.org/entity/Q1079023'), ('objname', 'championship')]
[('obj', 'http://www.wikidata.org/entity/Q500834'), ('objname', 'tournament')]
3


In [198]:
#[('obj', 'http://www.wikidata.org/entity/Q1079023'), ('objname', 'championship')]

queryString = """
SELECT DISTINCT  ?obj ?objname
WHERE {
   # bind something
   #wd:Q11571 wdt:P118 ?o.
   ?obj wdt:P31 wd:Q1079023 .
   
   #?o sc:name ?name.
   ?obj sc:name ?objname.
   FILTER ( REGEX(?objname, ".*UEFA.*"))
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q22097855'), ('objname', "2017 UEFA Women's Under-19 Championship qualification")]
[('obj', 'http://www.wikidata.org/entity/Q28128270'), ('objname', "2018 UEFA Women's Under-19 Championship qualification")]
[('obj', 'http://www.wikidata.org/entity/Q42916141'), ('objname', "2019 UEFA Women's Under-19 Championship qualification")]
[('obj', 'http://www.wikidata.org/entity/Q182068'), ('objname', 'UEFA Intertoto Cup')]
[('obj', 'http://www.wikidata.org/entity/Q16958545'), ('objname', "2014 UEFA Women's Under-19 Championship First qualifying round")]
[('obj', 'http://www.wikidata.org/entity/Q60526651'), ('objname', '2019 UEFA Under-19 Futsal Championship qualification')]
[('obj', 'http://www.wikidata.org/entity/Q30633478'), ('objname', 'UEFA Under-19 Futsal Championship')]
[('obj', 'http://www.wikidata.org/entity/Q48839484'), ('objname', "2019 UEFA Women's Under-17 Championship qualification")]
[('obj', 'http://www.wikidata.org/entity/Q6052666

In [204]:
#[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team')]
queryString = """
SELECT DISTINCT  ?obj ?objname ?s ?sname
WHERE {
   # bind something
   wd:Q11571 wdt:P54 ?obj.
   #?obj wdt:P31 wd:Q1079023 .
   #wd:Q1079023 wdt:P279 ?s .
   
   #?o sc:name ?name.
   ?obj sc:name ?objname.
   #?s sc:name ?sname.
   #FILTER ( REGEX(?sname, ".*UEFA.*"))
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3590758'), ('objname', 'Portugal Olympic football team')]
[('obj', 'http://www.wikidata.org/entity/Q1422'), ('objname', 'Juventus F.C.')]
[('obj', 'http://www.wikidata.org/entity/Q18656'), ('objname', 'Manchester United F.C.')]
[('obj', 'http://www.wikidata.org/entity/Q8682'), ('objname', 'Real Madrid CF')]
[('obj', 'http://www.wikidata.org/entity/Q1630430'), ('objname', 'Portugal national under-21 football team')]
[('obj', 'http://www.wikidata.org/entity/Q267245'), ('objname', 'Portugal national association football team')]
[('obj', 'http://www.wikidata.org/entity/Q75729'), ('objname', 'Sporting CP')]
[('obj', 'http://www.wikidata.org/entity/Q1772776'), ('objname', 'Portugal national under-20 football team')]
[('obj', 'http://www.wikidata.org/entity/Q3590754'), ('objname', 'Portugal national under-17 football team')]
[('obj', 'http://www.wikidata.org/entity/Q21079208'), ('objname', 'Portugal national under-15 football team')]
10


In [208]:
#[('obj', 'http://www.wikidata.org/entity/Q8682'), ('objname', 'Real Madrid CF')]
queryString = """
SELECT DISTINCT  ?p ?pname ?o ?oname
WHERE {
   # bind something
   
   wd:Q8682 ?p ?o .
   
   
   ?o sc:name ?oname.
   ?p sc:name ?pname.
   FILTER ( REGEX(?oname, ".*UEFA.*"))
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pname', 'victory'), ('o', 'http://www.wikidata.org/entity/Q20631590'), ('oname', '2017 UEFA Super Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pname', 'victory'), ('o', 'http://www.wikidata.org/entity/Q18740961'), ('oname', '2016–17 UEFA Champions League')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pname', 'victory'), ('o', 'http://www.wikidata.org/entity/Q948609'), ('oname', '2002 UEFA Super Cup')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pname', 'victory'), ('o', 'http://www.wikidata.org/entity/Q30608'), ('oname', '2013–14 UEFA Champions League')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pname', 'victory'), ('o', 'http://www.wikidata.org/entity/Q241409'), ('oname', '1997–98 UEFA Champions League')]
[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pname', 'victory'), ('o', 'http://www.wikidata.org/entity/Q754140'), ('oname', '1985–86 UEFA Cup')]
[('p', 'http:/

Final query for this task

In [ ]:
#[('p', 'http://www.wikidata.org/prop/direct/P2522'), ('pname', 'victory'),
#('o', 'http://www.wikidata.org/entity/Q18740961'), ('oname', '2016–17 UEFA Champions League')]

queryString = """
SELECT DISTINCT   ?o ?oname
WHERE {
   # bind something
   
   wd:Q18740961 ?p ?o .
   
   
   ?o sc:name ?oname.

   FILTER ( REGEX(?oname, ".*UEFA.*"))
}
LIMIT 70
"""

print("Results")
x=run_query(queryString)


Results
[('o', 'http://www.wikidata.org/entity/Q24664024'), ('oname', '2016–17 UEFA Champions League qualifying phase and play-off round')]
[('o', 'http://www.wikidata.org/entity/Q18756'), ('oname', 'UEFA Champions League')]
[('o', 'http://www.wikidata.org/entity/Q20650267'), ('oname', 'Category:2016–17 UEFA Champions League')]
[('o', 'http://www.wikidata.org/entity/Q24685533'), ('oname', '2016–17 UEFA Champions League group stage')]
[('o', 'http://www.wikidata.org/entity/Q25239571'), ('oname', '2016–17 UEFA Champions League knockout phase')]
5


## Task 5
Consider the countries which have at least two different clubs of their nationality who won the UEFA Champions League. Return the list of these countries with the number of different clubs who won the UEFA Champions League (the result set must be triples of country IRI, label and #clubs).

In [20]:
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]

#FINDING THE COUNTRY OF EACH  association football club
queryString = """
SELECT DISTINCT  ?objname ?rname
WHERE {
    
    ?obj wdt:P31 wd:Q476028 ;
        wdt:P17 ?res.
    
    
    ?obj sc:name ?objname .
    ?res sc:name ?rname

   
}

LIMIT 10
"""
print("Results")
x=run_query(queryString)

Results
[('objname', 'Doğan Türk Birliği'), ('rname', 'Turkish Republic of Northern Cyprus')]
[('objname', 'Atlantico FC'), ('rname', 'Dominican Republic')]
[('objname', 'AS Pélican'), ('rname', 'Gabon')]
[('objname', 'Adouma FC'), ('rname', 'Gabon')]
[('objname', 'Delta Téléstar Gabon Télécom FC'), ('rname', 'Gabon')]
[('objname', 'Real de Banjul F.C.'), ('rname', 'The Gambia')]
[('objname', 'Horoya AC'), ('rname', 'Guinea')]
[('objname', 'Satellite FC'), ('rname', 'Guinea')]
[('objname', "Stella Club d'Adjamé"), ('rname', 'Ivory Coast')]
[('objname', 'Scorpion de Bey'), ('rname', 'Cameroon')]
10


In [14]:
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition'),
#('o', 'http://www.wikidata.org/entity/Q18756'), ('oname', 'UEFA Champions League')]
 #finding all the UEFA CHAMPION LEAGS 
queryString = """
SELECT DISTINCT  ?objname ?obj
WHERE {
    
    ?obj wdt:P3450 wd:Q18756 .
    
    ?obj sc:name ?objname .
    
   
}order by ?objname 

limit 100
"""
print("Results")
x=run_query(queryString)


Results
[('objname', '1955–56 European Cup'), ('obj', 'http://www.wikidata.org/entity/Q219384')]
[('objname', '1956–57 European Cup'), ('obj', 'http://www.wikidata.org/entity/Q245202')]
[('objname', '1957–58 European Cup'), ('obj', 'http://www.wikidata.org/entity/Q309318')]
[('objname', '1958–59 European Cup'), ('obj', 'http://www.wikidata.org/entity/Q309316')]
[('objname', '1959–60 European Cup'), ('obj', 'http://www.wikidata.org/entity/Q392535')]
[('objname', '1960–61 European Cup'), ('obj', 'http://www.wikidata.org/entity/Q330686')]
[('objname', '1961–62 European Cup'), ('obj', 'http://www.wikidata.org/entity/Q497028')]
[('objname', '1962–63 European Cup'), ('obj', 'http://www.wikidata.org/entity/Q500074')]
[('objname', '1963–64 European Cup'), ('obj', 'http://www.wikidata.org/entity/Q500084')]
[('objname', '1964–65 European Cup'), ('obj', 'http://www.wikidata.org/entity/Q497052')]
[('objname', '1965–66 European Cup'), ('obj', 'http://www.wikidata.org/entity/Q573225')]
[('objname', 

In [15]:
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition'),
#('o', 'http://www.wikidata.org/entity/Q18756'), ('oname', 'UEFA Champions League')]
 #THEY CHANGED THE NAME AFTER 1992 SO BEFORE THAT THE NAME WAS EUROPEAN CUP

#FINDING THE PROPERTIS OF THE LEAGS TO FIND THE WINNERS
queryString = """
SELECT DISTINCT  ?p ?name
WHERE {
    
    ?leags wdt:P3450 wd:Q18756 .
    
    ?leags ?p ?obj.
    
    ?p sc:name ?name .
    
   filter(REGEX(?name , 'winner'))
}

LIMIT 200
"""
print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
1


In [56]:
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition'),
#('o', 'http://www.wikidata.org/entity/Q18756'), ('oname', 'UEFA Champions League')]
#[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
 #THEY CHANGED THE NAME AFTER 1992 SO BEFORE THAT THE NAME WAS EUROPEAN CUP

#FINDING THE WINNERS' COUNTRY
queryString = """
SELECT DISTINCT  ?leags ?leagename ?winner ?winnername ?countryname 
WHERE {
    
    ?leags wdt:P3450 wd:Q18756 .
    
    ?leags wdt:P1346 ?winner.
    ?winner  wdt:P17 ?country.
    
    ?leags sc:name ?leagename .
    ?winner sc:name ?winnername.
    ?country sc:name ?countryname
    
   
}

LIMIT 10
"""
print("Results")
x=run_query(queryString)


Results
[('leags', 'http://www.wikidata.org/entity/Q275673'), ('leagename', '1992–93 UEFA Champions League'), ('winner', 'http://www.wikidata.org/entity/Q132885'), ('winnername', 'Olympique de Marseille'), ('countryname', 'France')]
[('leags', 'http://www.wikidata.org/entity/Q308996'), ('leagename', '2004–05 UEFA Champions League'), ('winner', 'http://www.wikidata.org/entity/Q1130849'), ('winnername', 'Liverpool F.C.'), ('countryname', 'United Kingdom')]
[('leags', 'http://www.wikidata.org/entity/Q30032467'), ('leagename', '2018–19 UEFA Champions League'), ('winner', 'http://www.wikidata.org/entity/Q1130849'), ('winnername', 'Liverpool F.C.'), ('countryname', 'United Kingdom')]
[('leags', 'http://www.wikidata.org/entity/Q641465'), ('leagename', '1980–81 European Cup'), ('winner', 'http://www.wikidata.org/entity/Q1130849'), ('winnername', 'Liverpool F.C.'), ('countryname', 'United Kingdom')]
[('leags', 'http://www.wikidata.org/entity/Q196681'), ('leagename', '1977–78 European Cup'), ('w

In [65]:
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition'),
#('o', 'http://www.wikidata.org/entity/Q18756'), ('oname', 'UEFA Champions League')]
#[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
 #THEY CHANGED THE NAME AFTER 1992 SO BEFORE THAT THE NAME WAS EUROPEAN CUP

#FINDING THE WINNERS' COUNTRY
queryString = """
SELECT DISTINCT  ?leagename ?winnername ?countryname 
WHERE {
    
    ?leags wdt:P3450 wd:Q18756 .
    
    ?leags wdt:P1346 ?winner.
    ?winner  wdt:P17 ?country.
    
    ?leags sc:name ?leagename .
    ?winner sc:name ?winnername.
    ?country sc:name ?countryname
    FILTER (REGEX(?countryname,'Italy'))
    
   
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)


Results
[('leagename', '1995–96 UEFA Champions League'), ('winnername', 'Juventus F.C.'), ('countryname', 'Italy')]
[('leagename', '1984–85 European Cup'), ('winnername', 'Juventus F.C.'), ('countryname', 'Italy')]
[('leagename', '2006–07 UEFA Champions League'), ('winnername', 'A.C. Milan'), ('countryname', 'Italy')]
[('leagename', '1993–94 UEFA Champions League'), ('winnername', 'A.C. Milan'), ('countryname', 'Italy')]
[('leagename', '1962–63 European Cup'), ('winnername', 'A.C. Milan'), ('countryname', 'Italy')]
[('leagename', '1988–89 European Cup'), ('winnername', 'A.C. Milan'), ('countryname', 'Italy')]
[('leagename', '1989–90 European Cup'), ('winnername', 'A.C. Milan'), ('countryname', 'Italy')]
[('leagename', '1968–69 European Cup'), ('winnername', 'A.C. Milan'), ('countryname', 'Italy')]
[('leagename', '2002–03 UEFA Champions League'), ('winnername', 'A.C. Milan'), ('countryname', 'Italy')]
[('leagename', '2009–10 UEFA Champions League'), ('winnername', 'FC Internazionale Mil

Final query for this task

In [17]:
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition'),
#('o', 'http://www.wikidata.org/entity/Q18756'), ('oname', 'UEFA Champions League')]
#[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
 #THEY CHANGED THE NAME AFTER 1992 SO BEFORE THAT THE NAME WAS EUROPEAN CUP

#FINAL QUERY
queryString = """
SELECT DISTINCT  ?country ?countryname (count(DISTINCT(?winner))as ?number)
WHERE {
    
    ?leags wdt:P3450 wd:Q18756 .
    
    ?leags wdt:P1346 ?winner.
    ?winner  wdt:P17 ?country.
    
    ?leags sc:name ?leagename .
    ?winner sc:name ?winnername.
    ?country sc:name ?countryname
    
   
}group by ?country ?countryname
having (count(DISTINCT (?winner))>1)

"""
print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryname', 'Portugal'), ('number', '2')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryname', 'Spain'), ('number', '2')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryname', 'Germany'), ('number', '3')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryname', 'Italy'), ('number', '3')]
[('country', 'http://www.wikidata.org/entity/Q55'), ('countryname', 'Netherlands'), ('number', '3')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryname', 'United Kingdom'), ('number', '6')]
6


## Task 6
Return for each european country the top level league (the result set must be a list of country IRI and label, and league IRI and label).

In [70]:
#[('res', 'http://www.wikidata.org/entity/Q15991303'), ('rname', 'association football league')]

#SEARCHING FOR A PROPERTY RELATED TO LEAGUE RANKINGS
queryString = """
SELECT DISTINCT  ?p ?name ?obj ?objname
WHERE {
    wd:Q15991303 ?p ?obj.
    
    ?p sc:name ?name .
    ?obj sc:name ?objname.
   
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)



Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P2499'), ('objname', 'league level above')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P2500'), ('objname', 'league level below')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P3983'), ('objname', 'sports league level')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q26887310'), ('objname', 'association football team season')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P17'), ('objname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), (

In [75]:
#[('o', 'http://www.wikidata.org/entity/Q182994'), ('name', 'Liga Portugal')]
#USING A FOUND NODE TO FIND THE PROPERTIES CONECTING THE LEAGS TO COUNTRIES AND RANKING 

queryString = """
SELECT DISTINCT  ?p ?name ?obj ?objname
WHERE {
   wd:Q182994 ?p ?obj.
   
   ?p sc:name ?name.
   ?obj sc:name ?objname .
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P2500'), ('name', 'league level below'), ('obj', 'http://www.wikidata.org/entity/Q618131'), ('objname', 'Portuguese Second Division')]
[('p', 'http://www.wikidata.org/prop/direct/P4794'), ('name', 'season starts'), ('obj', 'http://www.wikidata.org/entity/Q122'), ('objname', 'August')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q2736'), ('objname', 'association football')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country'), ('obj', 'http://www.wikidata.org/entity/Q45'), ('objname', 'Portugal')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q15991303'), ('objname', 'association football league')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class'), ('obj', 'http://www.wikidata.org/entity/Q31930761'), ('objname', "men's association football")]
[('p', '

In [87]:
#[('res', 'http://www.wikidata.org/entity/Q15991303'), ('rname', 'association football league')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'),
# ('obj', 'http://www.wikidata.org/entity/P2499'), ('objname', 'league level above')]
#[('p', 'http://www.wikidata.org/prop/direct/P2500'), ('name', 'league level below'),
#('obj', 'http://www.wikidata.org/entity/Q618131'), ('objname', 'Portuguese Second Division')]

#filtering top leages of every country 
queryString = """
SELECT DISTINCT   ?leage ?leagename ?country ?countryname
WHERE {
   ?leage wdt:P31 wd:Q15991303;
           wdt:P17 ?country.
    FILTER NOT EXISTS {?leage wdt:P2499 ?upper}
   
   ?leage sc:name ?leagename.
   ?country sc:name ?countryname .
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)


Results
[('leage', 'http://www.wikidata.org/entity/Q1104981'), ('leagename', 'Coca Cola GM'), ('country', 'http://www.wikidata.org/entity/Q223'), ('countryname', 'Greenland')]
[('leage', 'http://www.wikidata.org/entity/Q2180367'), ('leagename', 'Süper Lig'), ('country', 'http://www.wikidata.org/entity/Q23681'), ('countryname', 'Turkish Republic of Northern Cyprus')]
[('leage', 'http://www.wikidata.org/entity/Q2660485'), ('leagename', 'Primera División de Republica Dominicana'), ('country', 'http://www.wikidata.org/entity/Q786'), ('countryname', 'Dominican Republic')]
[('leage', 'http://www.wikidata.org/entity/Q338121'), ('leagename', 'Gabon Championnat National D1'), ('country', 'http://www.wikidata.org/entity/Q1000'), ('countryname', 'Gabon')]
[('leage', 'http://www.wikidata.org/entity/Q1007931'), ('leagename', 'GFA League First Division'), ('country', 'http://www.wikidata.org/entity/Q1005'), ('countryname', 'The Gambia')]
[('leage', 'http://www.wikidata.org/entity/Q1485381'), ('leage

In [92]:
#[('res', 'http://www.wikidata.org/entity/Q15991303'), ('rname', 'association football league')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'),
# ('obj', 'http://www.wikidata.org/entity/P2499'), ('objname', 'league level above')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
#[('c', 'http://www.wikidata.org/entity/Q46'), ('cname', 'Europe')]

#filtering european countries  
queryString = """
SELECT DISTINCT    ?country ?countryname ?leage ?leagename
WHERE {
   ?leage wdt:P31 wd:Q15991303;
           wdt:P17 ?country.
    ?country wdt:P30 wd:Q46.
    FILTER NOT EXISTS {?leage wdt:P2499 ?upper}
   
   ?leage sc:name ?leagename.
   ?country sc:name ?countryname .
} group by ?leage ?leagename ?country ?countryname
order by ?countryname

LIMIT 10
"""
print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q228'), ('countryname', 'Andorra'), ('leage', 'http://www.wikidata.org/entity/Q257282'), ('leagename', 'Primera Divisió')]
[('country', 'http://www.wikidata.org/entity/Q399'), ('countryname', 'Armenia'), ('leage', 'http://www.wikidata.org/entity/Q237181'), ('leagename', 'Armenian Premier League')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('leage', 'http://www.wikidata.org/entity/Q4825490'), ('leagename', 'Austrian Regional League Central')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('leage', 'http://www.wikidata.org/entity/Q25858144'), ('leagename', 'Landesliga West')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('leage', 'http://www.wikidata.org/entity/Q687207'), ('leagename', 'ÖFB-Frauenliga')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('leage', 'http://www.wikidata.org/entity/Q48254

In [97]:
#[('res', 'http://www.wikidata.org/entity/Q15991303'), ('rname', 'association football league')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'),
# ('obj', 'http://www.wikidata.org/entity/P2499'), ('objname', 'league level above')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
#[('c', 'http://www.wikidata.org/entity/Q46'), ('cname', 'Europe')]
#[ ('obj', 'http://www.wikidata.org/entity/P2500'), ('objname', 'league level below')]
#[ ('obj', 'http://www.wikidata.org/entity/P3983'), ('objname', 'sports league level')]

#AS THE PREVIOUS DID NOT WOTK PROPERLY CHECKING OTHER PROPERTIES  
queryString = """
SELECT DISTINCT    ?country ?countryname ?leage ?leagename
WHERE {
   ?leage wdt:P31 wd:Q15991303;
           wdt:P17 ?country.
    ?country wdt:P30 wd:Q46.
    FILTER NOT EXISTS {?leage wdt:P2499 ?upper}.
    FILTER EXISTS {?leage wdt:P2500 ?upper}
   
   ?leage sc:name ?leagename.
   ?country sc:name ?countryname .
} group by ?leage ?leagename ?country ?countryname
order by ?countryname

LIMIT 20
"""
print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q228'), ('countryname', 'Andorra'), ('leage', 'http://www.wikidata.org/entity/Q257282'), ('leagename', 'Primera Divisió')]
[('country', 'http://www.wikidata.org/entity/Q399'), ('countryname', 'Armenia'), ('leage', 'http://www.wikidata.org/entity/Q237181'), ('leagename', 'Armenian Premier League')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('leage', 'http://www.wikidata.org/entity/Q4825490'), ('leagename', 'Austrian Regional League Central')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('leage', 'http://www.wikidata.org/entity/Q219592'), ('leagename', 'Austrian Bundesliga')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('leage', 'http://www.wikidata.org/entity/Q4825492'), ('leagename', 'Austrian Regional League West')]
[('country', 'http://www.wikidata.org/entity/Q227'), ('countryname', 'Azerbaijan'), ('leage', 'http://www.wiki

In [100]:
#[('res', 'http://www.wikidata.org/entity/Q15991303'), ('rname', 'association football league')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'),
# ('obj', 'http://www.wikidata.org/entity/P2499'), ('objname', 'league level above')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
#[('c', 'http://www.wikidata.org/entity/Q46'), ('cname', 'Europe')]
#[ ('obj', 'http://www.wikidata.org/entity/P2500'), ('objname', 'league level below')]
#[ ('obj', 'http://www.wikidata.org/entity/P3983'), ('objname', 'sports league level')]

#AS THE PREVIOUS DID NOT WOTK CHECK WHAT ARE THE INSTANCES OF  sports league level
queryString = """
SELECT DISTINCT   ?obj 
WHERE {
   ?leage wdt:P31 wd:Q15991303.
   ?leage wdt:P3983 ?obj.
   
   #?obj sc:name ?name.
} 
LIMIT 100
"""
print("Results")
x=run_query(queryString)

#based on this we assume that level ones are the professional leagues (not sure!)
#to make sure check the assumption for the league serie A which we found the node befor 
#[('res', 'http://www.wikidata.org/entity/Q15804'), ('rname', 'Serie A')]


Results
[('obj', '2')]
[('obj', '4')]
[('obj', '1')]
[('obj', '5')]
[('obj', '7')]
[('obj', '3')]
[('obj', '6')]
[('obj', '8')]
[('obj', '9')]
[('obj', '10')]
10


In [101]:
#[('res', 'http://www.wikidata.org/entity/Q15804'), ('rname', 'Serie A')]
#proof of the ranking of the teams 
queryString = """
SELECT DISTINCT  ?obj
WHERE {
   wd:Q15804 wdt:P3983 ?obj.
   
   #?obj sc:name ?name.
}

LIMIT 100
"""
print("Results")
x=run_query(queryString)




Results
[('obj', '1')]
1


Final query for this task

In [19]:
#[('res', 'http://www.wikidata.org/entity/Q15991303'), ('rname', 'association football league')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type'),
# ('obj', 'http://www.wikidata.org/entity/P2499'), ('objname', 'league level above')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pname', 'continent')]
#[('c', 'http://www.wikidata.org/entity/Q46'), ('cname', 'Europe')]
#[ ('obj', 'http://www.wikidata.org/entity/P2500'), ('objname', 'league level below')]
#[ ('obj', 'http://www.wikidata.org/entity/P3983'), ('objname', 'sports league level')]

#The method I used to find the top leagues is :
#there should be no upper league 
#there should be leagues bellow 
#the sport level league should be 1 

#FINAL 
queryString = """
SELECT DISTINCT   ?country ?countryname  ?leage ?leagename
WHERE {
   ?leage wdt:P31 wd:Q15991303;
           wdt:P17 ?country;
           wdt:P3983 1 .
    ?country wdt:P30 wd:Q46.
    FILTER NOT EXISTS {?leage wdt:P2499 ?upper}.
    FILTER EXISTS {?leage wdt:P2500 ?lower}

    
   ?leage sc:name ?leagename.
   ?country sc:name ?countryname .
} group by  ?leagename  ?countryname
order by ?countryname

"""
print("Results")
x=run_query(queryString)


Results
[('country', 'http://www.wikidata.org/entity/Q228'), ('countryname', 'Andorra'), ('leage', 'http://www.wikidata.org/entity/Q257282'), ('leagename', 'Primera Divisió')]
[('country', 'http://www.wikidata.org/entity/Q399'), ('countryname', 'Armenia'), ('leage', 'http://www.wikidata.org/entity/Q237181'), ('leagename', 'Armenian Premier League')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('leage', 'http://www.wikidata.org/entity/Q219592'), ('leagename', 'Austrian Bundesliga')]
[('country', 'http://www.wikidata.org/entity/Q227'), ('countryname', 'Azerbaijan'), ('leage', 'http://www.wikidata.org/entity/Q326859'), ('leagename', 'Azerbaijan Premier League')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('countryname', 'Belarus'), ('leage', 'http://www.wikidata.org/entity/Q244464'), ('leagename', 'Belarusian Premier League')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryname', 'Belgium'), ('leage', 'http://www.wikidata.org/ent

## Task 7
Identify the BGP of the FIFA World Cup

In [131]:
#wd:Q11571	Cristiano Ronaldo	node
#checking the instances of each property
queryString = """
SELECT DISTINCT ?p ?name ?ins ?insname
WHERE {
   # bind something
   wd:Q11571 ?p ?obj .
   ?obj wdt:P31 ?ins.
   # get the label
   ?ins sc:name ?insname.
   ?p sc:name ?name
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received'), ('ins', 'http://www.wikidata.org/entity/Q618779'), ('insname', 'award')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('name', 'sex or gender'), ('ins', 'http://www.wikidata.org/entity/Q48264'), ('insname', 'gender identity')]
[('p', 'http://www.wikidata.org/prop/direct/P6379'), ('name', 'has works in the collection'), ('ins', 'http://www.wikidata.org/entity/Q3343298'), ('insname', 'non-departmental public body')]
[('p', 'http://www.wikidata.org/prop/direct/P54'), ('name', 'member of sports team'), ('ins', 'http://www.wikidata.org/entity/Q23759293'), ('insname', 'national Olympic football team')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed'), ('ins', 'http://www.wikidata.org/entity/Q1288568'), ('insname', 'modern language')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language'), ('ins', 'http://www.wikidata.org/en

In [133]:
#wd:Q11571	Cristiano Ronaldo	node
#checking the instances of each property
#[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q11571 wdt:P1344 ?p.
   
   # get the label
   ?p sc:name ?name
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q373501'), ('name', 'UEFA Euro 2020')]
[('p', 'http://www.wikidata.org/entity/Q102920'), ('name', 'UEFA Euro 2004')]
[('p', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q176883'), ('name', '2010 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q189571'), ('name', 'UEFA Euro 2016')]
[('p', 'http://www.wikidata.org/entity/Q22669'), ('name', 'UEFA Euro 2012')]
[('p', 'http://www.wikidata.org/entity/Q241864'), ('name', 'UEFA Euro 2008')]
[('p', 'http://www.wikidata.org/entity/Q324867'), ('name', 'La Liga')]
[('p', 'http://www.wikidata.org/entity/Q37285'), ('name', '2006 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q79859'), ('name', '2014 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q8558'), ('name', '2004 Summer Olympics')]
11


In [142]:
#wd:Q11571	Cristiano Ronaldo	node
#checking the instances of each property
#[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')
#[('p', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q11571 wdt:P1344 ?match.
   ?match ?p ?obj.
   
   
   # get the label
   ?p sc:name ?name
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4967'), ('name', 'prerequisite')]
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1448'), ('name', 'official name')]
[('p', 'http://www.wikida

In [143]:
#wd:Q11571	Cristiano Ronaldo	node
#checking the instances of each property
#[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')
#[('p', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q11571 wdt:P1344 ?match.
   ?match wdt:P31 ?p.
   
   
   # get the label
   ?p sc:name ?name
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q159821'), ('name', 'Summer Olympic Games')]
[('p', 'http://www.wikidata.org/entity/Q15991290'), ('name', 'professional sports league')]
[('p', 'http://www.wikidata.org/entity/Q27020041'), ('name', 'sports season')]
[('p', 'http://www.wikidata.org/entity/Q15991303'), ('name', 'association football league')]
[('p', 'http://www.wikidata.org/entity/Q107540719'), ('name', 'edition of the UEFA European Championship')]
5


In [146]:
#wd:Q11571	Cristiano Ronaldo	node
#checking the instances of each property
#[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')
#[('p', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
queryString = """
SELECT DISTINCT ?x ?name 
WHERE {
   # bind something
   wd:Q11571 wdt:P1344 ?match.
   ?match wdt:P279 ?p.
   ?x wdt:P31 ?p .

   
   FILTER (REGEX(?name, 'FIFA'))
   # get the label
   ?x sc:name ?name
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
Empty


Final query for this task

In [148]:
#wd:Q11571	Cristiano Ronaldo	node
#checking the instances of each property
#[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')
#[('p', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q11571 wdt:P1344 ?match.
   ?match wdt:P3450 ?p.
   
   
   # get the label
   ?p sc:name ?name
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q260858'), ('name', 'UEFA European Championship')]
2


## Task 8
Consider countries who participated at the FIFA World Cup more than 10 times. Return these countries with the number of apparitions (the result set must be triples of country IRI, label and #apparitions).

In [3]:
#wd:Q11571	Cristiano Ronaldo	node
#checking the  all instances of  FIFA WORLD CUPS
#[('p', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]


#
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   ?p wdt:P3450 wd:Q19317 .
   
   
   # get the label
   ?p sc:name ?name
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q101751'), ('name', '1994 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q131591'), ('name', '1938 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q132664'), ('name', '1970 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q48432'), ('name', '1930 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q170645'), ('name', '2018 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q176883'), ('name', '2010 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q37285'), ('name', '2006 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q46934'), ('name', '1982 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q46938'), ('name', '1986 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q46943'), ('name', '1978 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q47735'), ('name', '2002 FIFA World Cup')]
[('p', 'http://www.wikidata.org/entity/Q79859'), ('name', '2014 FIFA World Cup')]
[('

In [4]:
#wd:Q11571	Cristiano Ronaldo	node

#[('p', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]


#checking properties of matches for participents
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   ?match wdt:P3450 wd:Q19317 .
   ?match ?p ?obj .
   
   # get the label
   ?p sc:name ?name
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4967'), ('name', 'prerequisite')]
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1546'), ('name', 'motto')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P1617'), ('name', 'BBC Things ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'

In [27]:
#wd:Q11571	Cristiano Ronaldo	node
#[('p', 'http://www.wikidata.org/prop/direct/P710'), ('name', 'participant')]
#[('p', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]


#checking properties of matches for participents
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   ?match wdt:P3450 wd:Q19317 .
   ?match  wdt:P710 ?P .
   
   # get the label
   ?p sc:name ?name
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


#DID NOT WORK SO WE CHECK THE OTHER PROPERTIES

Results
[('p', 'http://www.wikidata.org/entity/P348'), ('name', 'software version identifier')]
[('p', 'http://www.wikidata.org/entity/P4669'), ('name', 'tabular software version')]
[('p', 'http://www.wikidata.org/entity/P747'), ('name', 'has edition or translation')]
[('p', 'http://www.wikidata.org/entity/Q1002728'), ('name', 'Fonsi Nieto')]
[('p', 'http://www.wikidata.org/entity/Q100326534'), ('name', "Al Stewart's albums in chronological order")]
[('p', 'http://www.wikidata.org/entity/Q100327667'), ('name', 'Category:Adelina Lopes Vieira')]
[('p', 'http://www.wikidata.org/entity/Q100343342'), ('name', 'Lubomír Stoklásek')]
[('p', 'http://www.wikidata.org/entity/Q1007576'), ('name', 'Ciudad Vieja')]
[('p', 'http://www.wikidata.org/entity/Q100879156'), ('name', 'Echániz')]
[('p', 'http://www.wikidata.org/entity/Q10100'), ('name', 'Natuna')]
[('p', 'http://www.wikidata.org/entity/Q101032298'), ('name', "Acid Mothers Temple's albums in chronological order")]
[('p', 'http://www.wikidata.

In [26]:
#wd:Q11571	Cristiano Ronaldo	node
#[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('name', 'participating team')]
#[('p', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]


#checking properties of matches for participents
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   ?match wdt:P3450 wd:Q19317 .
   ?match  wdt:P1923 ?p.
   
   # get the label
   ?p sc:name ?name
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q166196'), ('name', 'Poland national association football team')]
[('p', 'http://www.wikidata.org/entity/Q168397'), ('name', 'Romania national association football team')]
[('p', 'http://www.wikidata.org/entity/Q676899'), ('name', 'Italy national association football team')]
[('p', 'http://www.wikidata.org/entity/Q163534'), ('name', 'Austria national association football team')]
[('p', 'http://www.wikidata.org/entity/Q212564'), ('name', 'Colombia national football team')]
[('p', 'http://www.wikidata.org/entity/Q268208'), ('name', 'Australia national association football team')]
[('p', 'http://www.wikidata.org/entity/Q134916'), ('name', 'Uruguay national football team')]
[('p', 'http://www.wikidata.org/entity/Q181190'), ('name', 'Bosnia and Herzegovina national football team')]
[('p', 'http://www.wikidata.org/entity/Q170327'), ('name', 'South Africa national football team')]
[('p', 'http://www.wikidata.org/entity/Q186243'), ('name', 'Iraq n

In [28]:
#wd:Q11571	Cristiano Ronaldo	node
#[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('name', 'participating team')]
#[('p', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]

#checking properties of matches for participents
queryString = """
SELECT ?p ?name 
WHERE {
   # bind something
   ?match wdt:P3450 wd:Q19317 .
   ?match  wdt:P1923 ?team.
    ?team wdt:P17 ?p .
   # get the label
   ?p sc:name ?name
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1008'), ('name', 'Ivory Coast')]
[('p', 'http://www.wikidata.org/entity/Q1008'), ('name', 'Ivory Coast')]
[('p', 'http://www.wikidata.org/entity/Q1008'), ('name', 'Ivory Coast')]
[('p', 'http://www.wikidata.org/entity/Q1009'), ('name', 'Cameroon')]
[('p', 'http://www.wikidata.org/entity/Q1009'), ('name', 'Cameroon')]
[('p', 'http://www.wikidata.org/entity/Q1009'), ('name', 'Cameroon')]
[('p', 'http://www.wikidata.org/entity/Q1009'), ('name', 'Cameroon')]
[('p', 'http://www.wikidata.org/entity/Q1009'), ('name', 'Cameroon')]
[('p', 'http://www.wikidata.org/entity/Q1009'), ('name', 'Cameroon')]
[('p', 'http://www.wikidata.org/entity/Q1009'), ('name', 'Cameroon')]
[('p', 'http://www.wikidata.org/entity/Q1028'), ('name', 'Morocco')]
[('p', 'http://www.wikidata.org/entity/Q1028'), ('name', 'Morocco')]
[('p', 'http://www.wikidata.org/entity/Q1028'), ('name', 'Morocco')]
[('p', 'http://www.wikidata.org/entity/Q1028'), ('name', 'Morocco')]
[('p', 

In [11]:
#wd:Q11571	Cristiano Ronaldo	node
#[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('name', 'participating team')]
#[('p', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]

#checking properties of matches for participents
queryString = """
SELECT ?p ?name (count(?p)as ?number)
WHERE {
   # bind something
   ?match wdt:P3450 wd:Q19317 .
   ?match  wdt:P1923 ?team.
    ?team wdt:P17 ?p .
   # get the label
   ?p sc:name ?name
}group by ?p ?name
having (count(?p) >10)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('number', '18')]
[('p', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('number', '18')]
[('p', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('number', '21')]
[('p', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium'), ('number', '12')]
[('p', 'http://www.wikidata.org/entity/Q34'), ('name', 'Sweden'), ('number', '12')]
[('p', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain'), ('number', '15')]
[('p', 'http://www.wikidata.org/entity/Q414'), ('name', 'Argentina'), ('number', '17')]
[('p', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('number', '14')]
[('p', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('number', '27')]
[('p', 'http://www.wikidata.org/entity/Q77'), ('name', 'Uruguay'), ('number', '13')]
[('p', 'http://www.wikidata.org/entity/Q96'), ('name', 'Mexico'), ('number', '16')]
11


Final query for this task

In [20]:
#wd:Q11571	Cristiano Ronaldo	node
#[('p', 'http://www.wikidata.org/prop/direct/P1923'), ('name', 'participating team')]
#[('p', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]

#Final 
queryString = """
SELECT ?country ?name (count(?country)as ?number)
WHERE {
   # bind something
   ?match wdt:P3450 wd:Q19317 .
   ?match  wdt:P1923 ?team.
    ?team wdt:P17 ?country .
   # get the label
   ?country sc:name ?name
}group by ?country ?name
having (count(?country) >10)
order by desc(?number)
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('number', '27')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('number', '21')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('number', '18')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('number', '18')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('name', 'Argentina'), ('number', '17')]
[('country', 'http://www.wikidata.org/entity/Q96'), ('name', 'Mexico'), ('number', '16')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('name', 'Spain'), ('number', '15')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('number', '14')]
[('country', 'http://www.wikidata.org/entity/Q77'), ('name', 'Uruguay'), ('number', '13')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('name', 'Belgium'), ('number', '12')]
[('country', 'http://www.wikidata.org/entity/Q34'), ('name', 'Sweden'), ('n

## Task 9
Consider the countries that won the FIFA World Cup at least twice. Find the association football clubs of those countries won the UEFA Champions League (the result set must be couples of clubs IRI and label).

In [13]:
#wd:Q11571	Cristiano Ronaldo	node
#[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
#[('p', 'http://www.wikidata.org/entity/Q19317'), ('name', 'FIFA World Cup')]
#[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('name', 'sports season of league or competition')]
#[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]

#Finding the winners

queryString = """
SELECT ?country ?name (count(?country)as ?number)
WHERE {
   # bind something
   ?match wdt:P3450 wd:Q19317 .
   ?match  wdt:P1346 ?team.
    ?team wdt:P17 ?country .
   # get the label
   ?country sc:name ?name
}group by ?country ?name
having (count(?country) >1)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy'), ('number', '4')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('name', 'Germany'), ('number', '4')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('name', 'Brazil'), ('number', '5')]
[('country', 'http://www.wikidata.org/entity/Q414'), ('name', 'Argentina'), ('number', '2')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('number', '2')]
[('country', 'http://www.wikidata.org/entity/Q77'), ('name', 'Uruguay'), ('number', '2')]
6


In [15]:
# winners of the UEFA Champions League
queryString = """
SELECT DISTINCT  ?country ?countryname ?winnername
WHERE {
    
    ?leags wdt:P3450 wd:Q18756 .
    
    ?leags wdt:P1346 ?winner.
    ?winner  wdt:P17 ?country.
    
    ?leags sc:name ?leagename .
    ?winner sc:name ?winnername.
    ?country sc:name ?countryname
    
    
   
}group by ?country ?countryname ?winnername


LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryname', 'United Kingdom'), ('winnername', 'Chelsea F.C.')]
[('country', 'http://www.wikidata.org/entity/Q218'), ('countryname', 'Romania'), ('winnername', 'FCSB')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryname', 'Spain'), ('winnername', 'Real Madrid CF')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('countryname', 'Italy'), ('winnername', 'A.C. Milan')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryname', 'France'), ('winnername', 'Olympique de Marseille')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryname', 'United Kingdom'), ('winnername', 'Aston Villa F.C.')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('countryname', 'Kingdom of the Netherlands'), ('winnername', 'PSV Eindhoven')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryname', 'United Kingdom'), ('winnername', 'Liverpool F.C.')]
[('country', 'http://www.wikidata.org/entity

In [22]:
# winners of the UEFA Champions League
queryString = """
SELECT DISTINCT  ?country ?countryname ?winnername
WHERE {
    
    ?leags wdt:P3450 wd:Q18756 .
    
    ?leags wdt:P1346 ?winner.
    ?winner  wdt:P17 ?country.
    
    ?leags sc:name ?leagename .
    ?winner sc:name ?winnername.
    ?country sc:name ?countryname
    FILTER (REGEX(?countryname , 'Germany'))
    
    
   
}group by ?country ?countryname ?winnername


LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryname', 'Germany'), ('winnername', 'FC Bayern Munich')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryname', 'Germany'), ('winnername', 'Borussia Dortmund')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryname', 'Germany'), ('winnername', 'Hamburger SV')]
3


Final query for this task

In [23]:
# winners of the UEFA Champions League
queryString = """
SELECT DISTINCT  ?winner ?winnername
WHERE {
    
    ?leags wdt:P3450 wd:Q18756 .
    ?leags wdt:P1346 ?winner.
    ?winner  wdt:P17 ?countryouter.
    ?winner sc:name ?winnername.
    FILTER (?countryouter=?countryinner)
    {
        SELECT ?countryinner
        WHERE {
               # bind something
                ?match wdt:P3450 wd:Q19317 .
                ?match  wdt:P1346 ?team.
                ?team wdt:P17 ?countryinner .
            
            }group by ?countryinner ?name
        having (count(?countryinner) >1)
        
    }
   
}


LIMIT 100
"""
print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q132885'), ('winnername', 'Olympique de Marseille')]
[('winner', 'http://www.wikidata.org/entity/Q15789'), ('winnername', 'FC Bayern Munich')]
[('winner', 'http://www.wikidata.org/entity/Q41420'), ('winnername', 'Borussia Dortmund')]
[('winner', 'http://www.wikidata.org/entity/Q51974'), ('winnername', 'Hamburger SV')]
[('winner', 'http://www.wikidata.org/entity/Q1422'), ('winnername', 'Juventus F.C.')]
[('winner', 'http://www.wikidata.org/entity/Q1543'), ('winnername', 'A.C. Milan')]
[('winner', 'http://www.wikidata.org/entity/Q631'), ('winnername', 'FC Internazionale Milano')]
7
